## Comparing MLP with a Pytorch fully connected network

### Models

In [1]:
import numpy as np
import torch
from MLP import MLP

# Hyper parameters
batch_sz = 1
dim_in = 4
dim_out = 2
dim_hidden = 10
N_layers = 2
learning_rate = 1e-3

# Network with Pytorch (3 layers)
torchnet = torch.nn.Sequential(
    torch.nn.Linear(dim_in, dim_hidden),
    torch.nn.Sigmoid(),
    torch.nn.Linear(dim_hidden, dim_out),
    torch.nn.Sigmoid())

# Network with MLP.py
MLP = MLP(N_layers, dim_hidden, dim_in, dim_out)

### Training

In [2]:
# Input / Output
x = torch.rand(batch_sz, dim_in)
y = torch.rand(batch_sz, dim_out)

# MSE loss function
loss_fn = torch.nn.MSELoss()

# Training torchnet
epochs = 1000
for epoch in range(epochs):
    y_hat = torchnet(x)      # Feedforward : estimation of the output with the model
    loss = loss_fn(y_hat, y)
    
    torchnet.zero_grad() # Set gradients to null tensor
    loss.backward()   # Backward propagation

    with torch.no_grad(): # Disables the gradient calculation
        for param in torchnet.parameters():
            param -= learning_rate * param.grad # Gradient descent

### Test

In [3]:
# Test sample
x_test = torch.rand(batch_sz, dim_in)

# Estimation
y_hat = torchnet.forward(x_test)
print("C0 : {:.4f}\nC1 : {:.4f}\n\n".format(y_hat[0][0], y_hat[0][1]))

C0 : 0.5624
C1 : 0.6877


